In [1]:
import numpy as np
import pandas as pd
import rasterio
import matplotlib.pyplot as plt

In [2]:
lat = np.array([49.9166666666664 - i * 0.0416666666667 for i in range(357)])
lon = np.array([-105.0416666666507 + i * 0.0416666666667 for i in range(722)])
Lon, Lat = np.meshgrid(lon, lat)

am = rasterio.open('PRISM_tmin_stable_4kmD2_19810101_bil.bil')
a = am.read()[0, :357, 479:1201]
mask = np.where(a>-1000, 1, 0)

tmin81 = np.load('./prism/mw_tmin/1981_tmin.npz')['tmin']
t, y, x = tmin81.shape
y, x

(357, 722)

In [18]:
# MonStart = [59, 90, 120]
# MonStartLeap = [60, 91, 121]
# MonDays = [31, 30, 31]
# Tcriteria = [0, -5, -10, -15, -20]
# March = np.zeros((38, 5, 31))
# April = np.zeros((38, 5, 30))
# May = np.zeros((38, 5, 31))
# count = [March, April, May]

# for yl in range(38):
#     tmin = np.load(f'./prism/mw_tmin/{1981+yl}_tmin.npz')['tmin']
#     tmin = np.where(tmin>-100, tmin, np.nan)
#     if (1981+yl)%4:
#         MonS = MonStart
#     else:
#         MonS = MonStartLeap
#     for mi in range(3):
#         for ti in range(5):
#             count[mi][yl, ti, :] = np.array([np.where(tmin[MonS[mi]+di, :, :]<Tcriteria[ti], 1, 0).sum()/(y*x) for di in range(MonDays[mi])])

# np.save('./var/MarchCount', count[0])
# np.save('./var/AprilCount', count[1])
# np.save('./var/MayCount', count[2])

In [31]:
# states = ['MI', 'ND', 'SD', 'NE', 'KS', 'OK', 'MN', 'IA', 'MO', 'WI', 'IL', 'IN', 'OH', 'KY', 'WV', 'PA']
# mdmask = [np.load(f'./var/{states[ii]}_mask.npy') for ii in range(16)]

# MonStart = [59, 90, 120]
# MonStartLeap = [60, 91, 121]
# MonDays = [31, 30, 31]
# Tcriteria = [0, -5, -10, -15, -20]
# March = np.zeros((38, 5, 16, 31))
# April = np.zeros((38, 5, 16, 30))
# May = np.zeros((38, 5, 16, 31))
# count = [March, April, May]

# for yl in range(38):
#     tmin = np.load(f'./prism/mw_tmin/{1981+yl}_tmin.npz')['tmin']
#     tmin = np.where(tmin>-100, tmin, np.nan)
#     if (1981+yl)%4:
#         MonS = MonStart
#     else:
#         MonS = MonStartLeap
#     for mi in range(3):
#         for ti in range(5):
#             for si in range(16):
#                 count[mi][yl, ti, si, :] = np.array([np.where(np.where(mdmask[si], tmin[MonS[mi]+di, :, :], np.nan)<Tcriteria[ti], 1, 0).sum() / mdmask[si].sum() for di in range(MonDays[mi])])

# np.save('./var/MarchCountState', count[0])
# np.save('./var/AprilCountState', count[1])
# np.save('./var/MayCountState', count[2])

<ipython-input-31-8bd70ce3b233>:23: RuntimeWarning: invalid value encountered in less
  count[mi][yl, ti, si, :] = np.array([np.where(np.where(mdmask[si], tmin[MonS[mi]+di, :, :], np.nan)<Tcriteria[ti], 1, 0).sum() / mdmask[si].sum() for di in range(MonDays[mi])])


In [20]:
March = np.load('./var/MarchCount.npy')
April = np.load('./var/AprilCount.npy')
May = np.load('./var/MayCount.npy')

In [21]:
fig = plt.figure(figsize=(8, 6))
X = np.arange(1981, 2019)
plt.plot(X, March[:, 0, :].sum(axis=1), 'b-o', lw=1, ms=3)
plt.plot(X, March[:, 1, :].sum(axis=1), 'g-o', lw=1, ms=3)
plt.plot(X, March[:, 2, :].sum(axis=1), 'y-o', lw=1, ms=3)
plt.plot(X, March[:, 3, :].sum(axis=1), 'm-o', lw=1, ms=3)
plt.plot(X, March[:, 4, :].sum(axis=1), 'c-o', lw=1, ms=3)
plt.xticks(np.arange(1981, 2018, 3), [str(X[i]) for i in range(0, 38, 3)])
plt.ylim(0, 25)
plt.legend(['Tmin<0\u2103', 'Tmin<-5\u2103', 'Tmin<-10\u2103', 'Tmin<-15\u2103', 'Tmin<-20\u2103'], frameon=False)
plt.title('31 days Sum of Area Percentage in March', fontsize='large', fontweight='bold')
plt.savefig('./plot/Composite/MarchTotal.png', bbox_inches='tight')

In [22]:
fig = plt.figure(figsize=(8, 6))
X = np.arange(1981, 2019)
plt.plot(X, April[:, 0, :].sum(axis=1), 'b-o', lw=1, ms=3)
plt.plot(X, April[:, 1, :].sum(axis=1), 'g-o', lw=1, ms=3)
plt.plot(X, April[:, 2, :].sum(axis=1), 'y-o', lw=1, ms=3)
plt.plot(X, April[:, 3, :].sum(axis=1), 'm-o', lw=1, ms=3)
plt.plot(X, April[:, 4, :].sum(axis=1), 'c-o', lw=1, ms=3)
plt.xticks(np.arange(1981, 2018, 3), [str(X[i]) for i in range(0, 38, 3)])
plt.ylim(0, 15)
plt.legend(['Tmin<0\u2103', 'Tmin<-5\u2103', 'Tmin<-10\u2103', 'Tmin<-15\u2103', 'Tmin<-20\u2103'], frameon=False)
plt.title('30 days Sum of Area Percentage in April', fontsize='large', fontweight='bold')
plt.savefig('./plot/Composite/AprilTotal.png', bbox_inches='tight')

In [23]:
fig = plt.figure(figsize=(8, 6))
X = np.arange(1981, 2019)
plt.plot(X, May[:, 0, :].sum(axis=1), 'b-o', lw=1, ms=3)
plt.plot(X, May[:, 1, :].sum(axis=1), 'g-o', lw=1, ms=3)
plt.plot(X, May[:, 2, :].sum(axis=1), 'y-o', lw=1, ms=3)
plt.plot(X, May[:, 3, :].sum(axis=1), 'm-o', lw=1, ms=3)
plt.plot(X, May[:, 4, :].sum(axis=1), 'c-o', lw=1, ms=3)
plt.xticks(np.arange(1981, 2018, 3), [str(X[i]) for i in range(0, 38, 3)])
plt.ylim(0, 4)
plt.legend(['Tmin<0\u2103', 'Tmin<-5\u2103', 'Tmin<-10\u2103', 'Tmin<-15\u2103', 'Tmin<-20\u2103'], frameon=False)
plt.title('31 days Sum of Area Percentage in May', fontsize='large', fontweight='bold')
plt.savefig('./plot/Composite/MayTotal.png', bbox_inches='tight')

In [3]:
MarchS = np.load('./var/MarchCountState.npy')
AprilS = np.load('./var/AprilCountState.npy')
MayS = np.load('./var/MayCountState.npy')

In [25]:
states = ['MI', 'ND', 'SD', 'NE', 'KS', 'OK', 'MN', 'IA', 'MO', 'WI', 'IL', 'IN', 'OH', 'KY', 'WV', 'PA']
fig, axs = plt.subplots(4, 4, sharex=True, sharey=True, figsize=(16, 14))
for i in range(16):
    plt.subplot(4,4,i+1)
    X = np.arange(1981, 2019)
    plt.plot(X, MarchS[:, 0, i, :].sum(axis=1), 'b-o', lw=1, ms=3)
    plt.plot(X, MarchS[:, 1, i, :].sum(axis=1), 'g-o', lw=1, ms=3)
    plt.plot(X, MarchS[:, 2, i, :].sum(axis=1), 'y-o', lw=1, ms=3)
    plt.plot(X, MarchS[:, 3, i, :].sum(axis=1), 'm-o', lw=1, ms=3)
    plt.plot(X, MarchS[:, 4, i, :].sum(axis=1), 'c-o', lw=1, ms=3)
    plt.xticks(np.arange(1981, 2019, 4), [str(X[i])[2:] for i in range(0, 39, 4)])
    plt.ylim(0, 32)
    plt.grid()
    plt.title(f'March in {states[i]}', fontweight='bold')
    if not i%4:
        plt.legend(['Tmin<0\u2103', 'Tmin<-5\u2103', 'Tmin<-10\u2103', 'Tmin<-15\u2103', 'Tmin<-20\u2103'], frameon=False)
plt.savefig('./plot/Composite/MarchState.png', bbox_inches='tight')

In [26]:
states = ['MI', 'ND', 'SD', 'NE', 'KS', 'OK', 'MN', 'IA', 'MO', 'WI', 'IL', 'IN', 'OH', 'KY', 'WV', 'PA']
fig, axs = plt.subplots(4, 4, sharex=True, sharey=True, figsize=(16, 14))
for i in range(16):
    plt.subplot(4,4,i+1)
    X = np.arange(1981, 2019)
    plt.plot(X, AprilS[:, 0, i, :].sum(axis=1), 'b-o', lw=1, ms=3)
    plt.plot(X, AprilS[:, 1, i, :].sum(axis=1), 'g-o', lw=1, ms=3)
    plt.plot(X, AprilS[:, 2, i, :].sum(axis=1), 'y-o', lw=1, ms=3)
    plt.plot(X, AprilS[:, 3, i, :].sum(axis=1), 'm-o', lw=1, ms=3)
    plt.plot(X, AprilS[:, 4, i, :].sum(axis=1), 'c-o', lw=1, ms=3)
    plt.xticks(np.arange(1981, 2019, 4), [str(X[i])[2:] for i in range(0, 39, 4)])
    plt.ylim(0, 32)
    plt.grid()
    plt.title(f'April in {states[i]}', fontweight='bold')
    if not i%4:
        plt.legend(['Tmin<0\u2103', 'Tmin<-5\u2103', 'Tmin<-10\u2103', 'Tmin<-15\u2103', 'Tmin<-20\u2103'], frameon=False)
plt.savefig('./plot/Composite/AprilState.png', bbox_inches='tight')

In [27]:
states = ['MI', 'ND', 'SD', 'NE', 'KS', 'OK', 'MN', 'IA', 'MO', 'WI', 'IL', 'IN', 'OH', 'KY', 'WV', 'PA']
fig, axs = plt.subplots(4, 4, sharex=True, sharey=True, figsize=(16, 14))
for i in range(16):
    plt.subplot(4,4,i+1)
    X = np.arange(1981, 2019)
    plt.plot(X, MayS[:, 0, i, :].sum(axis=1), 'b-o', lw=1, ms=3)
    plt.plot(X, MayS[:, 1, i, :].sum(axis=1), 'g-o', lw=1, ms=3)
    plt.plot(X, MayS[:, 2, i, :].sum(axis=1), 'y-o', lw=1, ms=3)
    plt.plot(X, MayS[:, 3, i, :].sum(axis=1), 'm-o', lw=1, ms=3)
    plt.plot(X, MayS[:, 4, i, :].sum(axis=1), 'c-o', lw=1, ms=3)
    plt.xticks(np.arange(1981, 2019, 4), [str(X[i])[2:] for i in range(0, 39, 4)])
    plt.ylim(0, 10)
    plt.grid()
    plt.title(f'May in {states[i]}', fontweight='bold')
    if not i%4:
        plt.legend(['Tmin<0\u2103', 'Tmin<-5\u2103', 'Tmin<-10\u2103', 'Tmin<-15\u2103', 'Tmin<-20\u2103'], frameon=False)
plt.savefig('./plot/Composite/MayState.png', bbox_inches='tight')